In [1]:
from TTS.api import TTS

In [29]:
import noisereduce as nr
import librosa
import soundfile as sf

In [17]:
import os
import torch

In [4]:
# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"

# List available 🐸TTS models
print(TTS().list_models())

In [5]:
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

 > You must confirm the following:
 | > "I have purchased a commercial license from Coqui: licensing@coqui.ai"
 | > "Otherwise, I agree to the terms of the non-commercial CPML: https://coqui.ai/cpml" - [y/n]
 > Downloading model to /Users/aryanmishra/Library/Application Support/tts/tts_models--multilingual--multi-dataset--xtts_v2


100%|██████████| 1.87G/1.87G [02:25<00:00, 12.9MiB/s]
100%|██████████| 4.37k/4.37k [00:00<00:00, 10.4kiB/s]
100%|██████████| 361k/361k [00:00<00:00, 805kiB/s]
100%|██████████| 32.0/32.0 [00:00<00:00, 58.3iB/s]
 79%|███████▉  | 6.16M/7.75M [00:00<00:00, 9.51MiB/s]/Users/aryanmishra/Desktop/HyperLang/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 > Model's license - CPML
 > Check https://coqui.ai/cpml.txt for more info.


100%|██████████| 7.75M/7.75M [00:13<00:00, 9.51MiB/s]

 > Using model: xtts


/Users/aryanmishra/Desktop/HyperLang/.venv/lib/python3.9/site-packages/TTS/tts/layers/xtts/xtts_manager.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.speakers = t

In [12]:
sample_audio_file = "./data/reference_audios/medellin_sample_1.wav"

In [13]:
tts.tts_to_file(text="¡Claro, qué deseas probar hoy? Tenemos un burrito con carne de res, un arepa con queso y una torta de guayaba.", speaker_wav=sample_audio_file, language="es", file_path="coqui_output.wav")

 > Text splitted to sentences.
['¡Claro, qué deseas probar hoy?', 'Tenemos un burrito con carne de res, un arepa con queso y una torta de guayaba.']
 > Processing time: 26.552266120910645
 > Real-time factor: 1.7298072113018808


'coqui_output.wav'

### Let's try a real dialogue 

In [19]:
def synthesize_speech(text, speaker_wav, output_path):
    # device = "cuda" if torch.cuda.is_available() else "cpu"
    # tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)
    tts.tts_to_file(
        text=text,
        file_path=output_path,
        speaker_wav=speaker_wav,
        language="es" if "Maestro" not in speaker_wav else "en"
    )

In [20]:
speaker_refs = {
    "Juan Pablo": "./data/reference_audios/medellin_sample_2.wav",
    "Ana": "./data/reference_audios/medellin_sample_1.wav",
    "Maestro": "./data/reference_audios/aryan_sample_1.wav"
}

In [24]:
dialogue = [
    'Juan Pablo: Hola, ¿puedes darme alguna recomendación sobre lo que ordenar?',
    'Ana: ¡Claro, qué deseas probar hoy? Tenemos un burrito con carne de res, un arepa con queso y una torta de guayaba.',
    'Maestro: Juan Pablo is asking for a recommendation on what to order. Ana is offering three options: a beef burrito, a cheese arepa, and a guava tart.',
    'Juan Pablo: Eso sí, ¿cómo es el burrito con carne?',
    'Ana: Es uno de nuestros platos más populares. Tiene carne de res, arroz, frijoles negros y salsa verde.',
    "Maestro: Juan Pablo wants to know about the beef burrito. Ana explains that it's one of their most popular dishes, containing beef, rice, black beans, and green sauce.",
    'Juan Pablo: Perfecto, lo tomaré. ¿Y si me lo traes con un refresco de piña?',
    'Ana: Claro, nosotros también tenemos un refresco de maracuyá. ¿Te gustaría probarlo?',
    'Juan Pablo: ¡Claro! Lo pediré, entonces.',
    'Ana: Muy bien. ¡Gracias por elegir nuestro café!',
    "Maestro: Juan Pablo orders the beef burrito with a pineapple drink and also asks for a taste of the maracuyá drink. Ana confirms the order and thanks him for choosing their café.",
    'Ana: Aquí está tu pedido, señor Juan Pablo. Disfruta!',
    'Juan Pablo: Muchas gracias, Ana. ¡Espero volver pronto!',
    'Ana: Es un placer tenerte aquí, ¡vuelve siempre!',
    'Maestro: Juan Pablo now starts eating and enjoying his order. Once he finishes, he gets up and leaves the place.'
]

In [25]:
output_dir = "./data/output_audios/"
os.makedirs(output_dir, exist_ok=True)

audio_files = []

for i, line in enumerate(dialogue):
    speaker, text = line.split(": ", 1)
    output_path = os.path.join(output_dir, f"line_{i+1}.wav")
    synthesize_speech(text, speaker_refs[speaker], output_path)
    audio_files.append(output_path)

 > Text splitted to sentences.
['Hola, ¿puedes darme alguna recomendación sobre lo que ordenar?']
 > Processing time: 13.19713807106018
 > Real-time factor: 3.0799840650600863
 > Text splitted to sentences.
['¡Claro, qué deseas probar hoy?', 'Tenemos un burrito con carne de res, un arepa con queso y una torta de guayaba.']
 > Processing time: 40.171984910964966
 > Real-time factor: 1.8403702116024065
 > Text splitted to sentences.
['Juan Pablo is asking for a recommendation on what to order.', 'Ana is offering three options: a beef burrito, a cheese arepa, and a guava tart.']
 > Processing time: 23.0907199382782
 > Real-time factor: 1.8013075067892925
 > Text splitted to sentences.
['Eso sí, ¿cómo es el burrito con carne?']
 > Processing time: 8.707942962646484
 > Real-time factor: 2.541968628552677
 > Text splitted to sentences.
['Es uno de nuestros platos más populares.', 'Tiene carne de res, arroz, frijoles negros y salsa verde.']
 > Processing time: 19.84798574447632
 > Real-time f

### Audio Post Processing

The current audio samples (look into output_audios line by line wav files) are still a bit noisy. Let's attempt to clean it up a bit

In [26]:
def reduce_noise(input_path, output_path):
    audio_data, sample_rate = librosa.load(input_path, sr=None)
    reduced_noise = nr.reduce_noise(y=audio_data, sr=sample_rate)
    sf.write(output_path, reduced_noise, sample_rate)

In [62]:
current_dir_audios = "./data/output_audios/"
output_dir_cleaned = "./data/output_audios_cleaned_part_1/"
os.makedirs(output_dir_cleaned, exist_ok=True)  

for i in range(1, 16):  
    input_file = os.path.join(current_dir_audios, f"line_{i}.wav")
    output_file = os.path.join(output_dir_cleaned, f"line_{i}_cleaned.wav")
    
    if os.path.exists(input_file): 
        print(f"Processing {input_file}...")
        reduce_noise(input_file, output_file)
    else:
        print(f"File {input_file} does not exist!")

Processing ./data/output_audios/line_1.wav...
Processing ./data/output_audios/line_2.wav...
Processing ./data/output_audios/line_3.wav...
Processing ./data/output_audios/line_4.wav...
Processing ./data/output_audios/line_5.wav...
Processing ./data/output_audios/line_6.wav...
Processing ./data/output_audios/line_7.wav...
Processing ./data/output_audios/line_8.wav...
Processing ./data/output_audios/line_9.wav...
Processing ./data/output_audios/line_10.wav...
Processing ./data/output_audios/line_11.wav...
Processing ./data/output_audios/line_12.wav...
Processing ./data/output_audios/line_13.wav...
Processing ./data/output_audios/line_14.wav...
Processing ./data/output_audios/line_15.wav...


### Problematic audio

Some audio files have this annoying background noise and weird AI voodo (like output_audios/line_2.wav). I think one way could be to use VAD to identify non-speech from speech. I started with some boilerplate code using claude 

In [67]:
import os
import webrtcvad
import librosa
import numpy as np
import soundfile as sf

# Create the output directory if it doesn't exist
output_dir = "./data/output_audios_cleaned_part_2"
os.makedirs(output_dir, exist_ok=True)

def identify_clean_voice(audio_path, output_dir):
    # Load the audio file
    audio_data, sample_rate = librosa.load(audio_path, sr=16000)  # Ensure 16 kHz sample rate
    audio_data = librosa.util.normalize(audio_data)  # Normalize audio

    # Convert to 16-bit PCM format required by webrtcvad
    audio_pcm = (audio_data * 32768).astype(np.int16)

    # Initialize WebRTC VAD
    vad = webrtcvad.Vad()
    vad.set_mode(3)  # Aggressive mode for detecting voice

    # Splitting audio into 10 ms chunks
    frame_size = int(sample_rate * 0.01)  # 10 ms
    chunks = [
        audio_pcm[i : i + frame_size]
        for i in range(0, len(audio_pcm), frame_size)
    ]

    # Identify which chunks contain voice
    clean_segments = []
    for chunk in chunks:
        if len(chunk) == frame_size and vad.is_speech(chunk.tobytes(), sample_rate):
            clean_segments.append(chunk)

    # Combine clean segments
    if clean_segments:
        clean_audio = np.concatenate(clean_segments, axis=0)
    else:
        clean_audio = np.array([], dtype=np.int16)  # Handle empty segments case

    # Create the output file path
    base_name = os.path.basename(audio_path).replace(".wav", "_clean_voice.wav")
    output_path = os.path.join(output_dir, base_name)

    # Save the clean audio
    sf.write(output_path, clean_audio / 32768, sample_rate)  # Normalize back to [-1, 1]
    print(f"Clean audio saved to {output_path}")

# we will process the de-noised versions first
input_dir = "./data/output_audios_cleaned_part_1"
audio_files = [f for f in os.listdir(input_dir) if f.endswith(".wav")]

for audio_file in audio_files:
    input_path = os.path.join(input_dir, audio_file)
    print(f"Processing {input_path}...")
    identify_clean_voice(input_path, output_dir)


Processing ./data/output_audios_cleaned_part_1/line_15_cleaned.wav...
Clean audio saved to ./data/output_audios_cleaned_part_2/line_15_cleaned_clean_voice.wav
Processing ./data/output_audios_cleaned_part_1/line_4_cleaned.wav...
Clean audio saved to ./data/output_audios_cleaned_part_2/line_4_cleaned_clean_voice.wav
Processing ./data/output_audios_cleaned_part_1/line_1_cleaned.wav...
Clean audio saved to ./data/output_audios_cleaned_part_2/line_1_cleaned_clean_voice.wav
Processing ./data/output_audios_cleaned_part_1/line_10_cleaned.wav...
Clean audio saved to ./data/output_audios_cleaned_part_2/line_10_cleaned_clean_voice.wav
Processing ./data/output_audios_cleaned_part_1/line_8_cleaned.wav...
Clean audio saved to ./data/output_audios_cleaned_part_2/line_8_cleaned_clean_voice.wav
Processing ./data/output_audios_cleaned_part_1/line_13_cleaned.wav...
Clean audio saved to ./data/output_audios_cleaned_part_2/line_13_cleaned_clean_voice.wav
Processing ./data/output_audios_cleaned_part_1/line_

In [81]:
from pydub import AudioSegment
import os
import re

# Directory containing the cleaned audio files
input_dir = "./data/output_audios_cleaned_part_2"
output_file = "./data/output_audios_combined.wav"

def combine_audio_with_pauses(input_dir, output_file, pause_duration_ms=800):
    """
    Combines all audio files in the input directory into one file with pauses between them.

    Args:
        input_dir (str): Directory containing cleaned audio files.
        output_file (str): Path to save the combined audio file.
        pause_duration_ms (int): Duration of the pause between audio files in milliseconds.
    """
    # Get a list of all audio files
    audio_files = [
        os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith(".wav")
    ]
    
    audio_files = sorted(
        audio_files,
        key=lambda x: int(re.search(r"line_(\d+)", os.path.basename(x)).group(1))
    )

    # Create a silent segment for the pause
    pause = AudioSegment.silent(duration=pause_duration_ms)

    # Initialize an empty audio segment
    combined_audio = AudioSegment.empty()

    # Process and concatenate each audio file with a pause
    for audio_file in audio_files:
        print(f"Adding {audio_file} to the combined audio...")
        audio_segment = AudioSegment.from_wav(audio_file)
        combined_audio += audio_segment + pause

    # Remove the final pause
    combined_audio = combined_audio[:-pause_duration_ms]

    # Export the combined audio file
    combined_audio.export(output_file, format="wav")
    print(f"Combined audio saved to {output_file}")

# Combine the audio files with a 500 ms pause
combine_audio_with_pauses(input_dir, output_file, pause_duration_ms=500)


Adding ./data/output_audios_cleaned_part_2/line_1_cleaned_clean_voice.wav to the combined audio...
Adding ./data/output_audios_cleaned_part_2/line_2_cleaned_clean_voice.wav to the combined audio...
Adding ./data/output_audios_cleaned_part_2/line_3_cleaned_clean_voice.wav to the combined audio...
Adding ./data/output_audios_cleaned_part_2/line_4_cleaned_clean_voice.wav to the combined audio...
Adding ./data/output_audios_cleaned_part_2/line_5_cleaned_clean_voice.wav to the combined audio...
Adding ./data/output_audios_cleaned_part_2/line_6_cleaned_clean_voice.wav to the combined audio...
Adding ./data/output_audios_cleaned_part_2/line_7_cleaned_clean_voice.wav to the combined audio...
Adding ./data/output_audios_cleaned_part_2/line_8_cleaned_clean_voice.wav to the combined audio...
Adding ./data/output_audios_cleaned_part_2/line_9_cleaned_clean_voice.wav to the combined audio...
Adding ./data/output_audios_cleaned_part_2/line_10_cleaned_clean_voice.wav to the combined audio...
Adding ./

In [82]:
# Combine the audio files with a 500 ms pause
combine_audio_with_pauses(input_dir, output_file, pause_duration_ms=500)

Adding ./data/output_audios_cleaned_part_2/line_1_cleaned_clean_voice.wav to the combined audio...
Adding ./data/output_audios_cleaned_part_2/line_2_cleaned_clean_voice.wav to the combined audio...
Adding ./data/output_audios_cleaned_part_2/line_3_cleaned_clean_voice.wav to the combined audio...
Adding ./data/output_audios_cleaned_part_2/line_4_cleaned_clean_voice.wav to the combined audio...
Adding ./data/output_audios_cleaned_part_2/line_5_cleaned_clean_voice.wav to the combined audio...
Adding ./data/output_audios_cleaned_part_2/line_6_cleaned_clean_voice.wav to the combined audio...
Adding ./data/output_audios_cleaned_part_2/line_7_cleaned_clean_voice.wav to the combined audio...
Adding ./data/output_audios_cleaned_part_2/line_8_cleaned_clean_voice.wav to the combined audio...
Adding ./data/output_audios_cleaned_part_2/line_9_cleaned_clean_voice.wav to the combined audio...
Adding ./data/output_audios_cleaned_part_2/line_10_cleaned_clean_voice.wav to the combined audio...
Adding ./